In [1]:
from prompt import *
from LLM import *

table_plan_map = {
    'company_info': 0,
    'company_register': 1,
    'sub_company_info': 2,
    'legal_document': 3,
    'legal_abstract': 4,
    'xzgxf_info': 5,
    'court_info': 6,
    'court_code': 7,
    'lawfirm_info': 8,
    'lawfirm_log': 9,
    'address_info': 10,
    'address_code': 11,
    'temp_info': 12
    }
prompt_list = [
    WRITER_PROMPT_0,
    WRITER_PROMPT_1,
    WRITER_PROMPT_2,
    WRITER_PROMPT_3,
    WRITER_PROMPT_4,
    WRITER_PROMPT_5,
    WRITER_PROMPT_6,
    WRITER_PROMPT_7,
    WRITER_PROMPT_8,
    WRITER_PROMPT_9,
    WRITER_PROMPT_10,
    WRITER_PROMPT_11,
    WRITER_PROMPT_12
]

In [2]:
question = "91310000677833266F的公司全称是？该公司的涉案次数为？（起诉日期在2020年）作为被起诉人的次数及总金额为？"
rsp = LLM(TABLE_PROMPT.format(question=question))
rsp

'```json\n["company_register", "legal_document"]\n```'

In [3]:
fcts = prase_json_from_response(rsp=rsp)
fcts

['company_register', 'legal_document']

In [4]:
plan_id = [table_plan_map[fct] for fct in fcts]
plan_id

[1, 3]

In [5]:
prompt = WRITER_PROMPT
for id in plan_id:
    prompt += prompt_list[id]

In [6]:
print(prompt)


# 任务 #
1.  如果问题可以被下方函数解决，请你先从functions模块中导入函数再解决任务，最后结果要使用print函数打印出来。
2.  请你分步执行代码。
3.  请你把代码放在```python```代码框中，方便我执行你的代码，把代码的运行结果用print方法打印出来。
4.  如果我执行的结果是正确的，请结合结果与问题返回相应的答案。

# 你可以使用的函数 #

get_company_register(company_name: str, need_fields: Optional[str] = None)
根据公司名称查询工商信息
need_fields可以选择的字段有[公司名称,登记状态,统一社会信用代码,法定代表人,注册资本,成立日期,企业地址,联系电话,联系邮箱,注册号,组织机构代码,参保人数,行业一级,行业二级,行业三级,曾用名,企业简介,经营范围]
例如：
    运行：print(get_company_register(company_name="天能电池集团股份有限公司", need_fields=["法定代表人", "联系电话"]))
    输出：{'法定代表人': '杨建芬', '联系电话': '0572-6029388'}

get_company_register_name(social_code: str)
根据统一社会信用代码查询公司名称
例如：
    运行：print(get_company_register_name(social_code="913305007490121183"))
    输出：{'公司名称': '天能电池集团股份有限公司'}

get_legal_document(legal_num: str, need_fields: Optional[str] = None)
根据案号查询裁判文书相关信息
need_fields可以选择的字段有[关联公司,标题,案号,文书类型,原告,被告,原告律师事务所,被告律师事务所,案由,涉案金额,判决结果,日期,文件名]
例如：
    运行：print(get_legal_document(legal_num="(2019)沪0115民初61975号", need_fields=["关联公司", "标题"]))
    输出：{'关联公司': '上海

In [7]:
utils_plan_map = {
    'get_sum': 0,
    'get_rank': 1,
    'save_dict_list_to_word': 2,
    'get_citizens_sue_citizens': 3,
    'get_company_sue_citizens': 4,
    'get_citizens_sue_company': 5,
    'get_company_sue_company': 6,
    }
utils_prompt = [
    GET_SUM,
    RANK,
    SAVE_DICT_LIST2WORD,
    CITIZENS_SUE_CITIZENS,
    COMPANY_SUE_CITIZENS,
    CITIZENS_SUE_COMPANY,
    COMPANY_SUE_COMPANY,
]

In [8]:
rsp = LLM(UTILS_PROMPT.format(question=question))
rsp

'```json\n["get_company_sue_company", "get_citizens_sue_company", "get_sum"]\n```'

In [9]:
fcts = prase_json_from_response(rsp=rsp)
fcts

['get_company_sue_company', 'get_citizens_sue_company', 'get_sum']

In [10]:
plan_id = [utils_plan_map[fct] for fct in fcts]
plan_id

[6, 5, 0]

In [11]:
for id in plan_id:
    prompt += utils_prompt[id]

In [12]:
write_execute(prompt=prompt, question=question)

code_executor_agent (to code_writer_agent):

91310000677833266F的公司全称是？该公司的涉案次数为？（起诉日期在2020年）作为被起诉人的次数及总金额为？

--------------------------------------------------------------------------------
code_writer_agent (to code_executor_agent):

为了解决这个问题，我们需要分步执行以下操作：

1. 使用`get_company_register_name`函数根据统一社会信用代码查询公司全称。
2. 使用`get_legal_document_list`函数查询公司的涉案次数。
3. 筛选起诉日期在2020年的记录，计算作为被起诉人的次数及总金额。

下面是相应的Python代码：

```python
from functions import get_company_register_name, get_legal_document_list

# 1. 查询公司全称
social_code = "91310000677833266F"
company_name = get_company_register_name(social_code)
print(f"公司全称：{company_name['公司名称']}")

# 2. 查询公司的涉案次数
documents = get_legal_document_list(company_name=company_name['公司名称'], need_fields=["标题", "案号", "涉案金额", "日期"])

# 3. 筛选2020年的记录，并计算作为被起诉人的次数及总金额
def is_defendant(title):
    # 简单判断标题中是否包含“与”字，作为是否为被告的依据
    return "与" in title

defendant_count = 0
total_defendant_amount = 0
for doc in documents:
    if doc['日期'].startswith('2020'):  # 筛选2020年的记录
     

'公司全称：上海晨光文具股份有限公司\n该公司的涉案次数：无法提供，信息中未包含整体涉案次数。\n作为被起诉人的次数：34次\n作为被起诉人的总金额：70,000.0元人民币'